In [2]:
# https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

import nltk
import pandas as pd

In [3]:
data_path = "C:\\Users\\arthur\\Documents\\Data Science Club\\cxc-2022\\NLP\\IMDB Dataset.csv"

raw_ds = pd.read_csv(data_path)
raw_ds.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
raw_ds["sentiment"].value_counts()

# perfectly balanced as all things should be

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [5]:
sentiment_dict = {
    "positive": 1,
    "negative": 0
}

raw_ds["sentiment_num"] = raw_ds["sentiment"].map(sentiment_dict)
raw_ds.head()

,review,sentiment,sentiment_num
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [6]:
import re

def cleanup(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'[^\w\s]', '', sentence)
    return sentence

cleanup(raw_ds["review"].iloc[1])

'a wonderful little production br br the filming technique is very unassuming very oldtimebbc fashion and gives a comforting and sometimes discomforting sense of realism to the entire piece br br the actors are extremely well chosen michael sheen not only has got all the polari but he has all the voices down pat too you can truly see the seamless editing guided by the references to williams diary entries not only is it well worth the watching but it is a terrificly written and performed piece a masterful production about one of the great masters of comedy and his life br br the realism really comes home with the little things the fantasy of the guard which rather than use the traditional dream techniques remains solid then disappears it plays on our knowledge and our senses particularly with the scenes concerning orton and halliwell and the sets particularly of their flat with halliwells murals decorating every surface are terribly well done'

In [7]:
import numpy as np

glove_path = "C:\\Users\\arthur\\Documents\\Data Science Club\\cxc-2022\\NLP\\glove.6B.50d.txt"
glove_dict = dict()
with open(glove_path,'r', encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], np.float32)
        glove_dict[word] = vector

In [8]:
words, mats = glove_dict.keys(), np.stack(list(glove_dict.values()))

In [9]:
from sklearn.decomposition import PCA

GLOVE_DIM = 15
pca = PCA(n_components=GLOVE_DIM)
mats_10 = pca.fit_transform(mats)

In [10]:
glove_dict_10 = dict()
for i, word in enumerate(words):
    glove_dict_10[word] = mats_10[i]

In [11]:
def vectorize_glove(word):
    if word in glove_dict.keys():
        return glove_dict_10[word]
    else:
        return np.random.random((GLOVE_DIM))
    
print("hello: {} \n\nkjnkejnv: {}".format(vectorize_glove("hello"), vectorize_glove("kjnkejnv")))

hello: [ 1.7657043   0.84778756  0.10259419 -1.4434046   0.56133264  0.70146716
 -1.2921506   0.73285055  0.25716665 -0.90811527 -0.2680069  -1.4289016
 -0.5981396  -0.01744122 -0.28990299] 

kjnkejnv: [0.10499276 0.45407643 0.01301038 0.80127699 0.28359563 0.7597087
 0.58672618 0.00362571 0.15237345 0.06864808 0.83499566 0.01235454
 0.96248535 0.63667105 0.85000484]


In [12]:
MAX_SENT_LEN = 30

def vectorize_sentence(sentence):
    sent_vec = []
    words = list(reversed(nltk.word_tokenize(sentence)))
    for i in range(MAX_SENT_LEN):
        if i < len(words):
            sent_vec.append(vectorize_glove(words[i]))
        else:
            sent_vec.append(np.zeros(GLOVE_DIM))
            
    return np.stack(sent_vec)

vectorize_sentence(cleanup("I love maths"))

array([[ 1.44100678,  0.14535385,  0.26128921,  0.15391552,  0.1783331 ,
         0.50999367, -0.3765927 ,  0.62655562,  0.65040094,  0.01854824,
         1.13450086,  0.64415216,  0.36923584,  0.02329123,  0.18185237],
       [ 3.9350462 ,  2.16337919,  0.17072383, -1.31969321,  0.37090284,
        -0.10306761, -0.2962755 ,  0.60151535,  0.62885267, -1.32612157,
         0.88642871, -0.73367554, -0.64391166, -0.13425593, -0.08219912],
       [ 4.72007322,  3.10054874, -0.14459926, -1.25010896,  0.73013687,
         0.60684705, -0.14994025,  0.4637996 ,  0.62890524, -0.63590807,
         0.99620849, -0.2821846 , -0.7947765 , -0.75747919, -0.5591796 ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        , 

In [13]:
def process_sentence(sentence):
    return vectorize_sentence(cleanup(sentence)).flatten()

In [14]:
from sklearn.model_selection import train_test_split
train_ds, test_ds = train_test_split(raw_ds, test_size=0.2, random_state=42)

x_train = train_ds["review"].apply(process_sentence)
x_train = np.stack(x_train.values)
y_train = train_ds["sentiment_num"].tolist()

x_test = test_ds["review"].apply(process_sentence)
x_test = np.stack(x_test.values)
y_test = test_ds["sentiment_num"].tolist()

In [15]:
np.sum(y_train)/len(y_train), np.sum(y_test)/len(y_test)

(0.499025, 0.5039)

In [16]:
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=50, max_depth=7)
model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=7, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=50, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [17]:
np.sum(model.predict(x_train) == y_train)/len(x_train)

0.90705

In [18]:
np.sum(model.predict(x_test) == y_test)/len(x_test)

0.6437